## <center>  CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2018 </center>

# <center> Optimizing Multi-core CPU Scheduling </center>

#### <center> Haoran Qiu (hqiu35@wisc.edu), Jian Wu (jwu384@wisc.com), Tao Ji (email address), and Yanting Liang (email address) </center>
#### <center> Instructor: Prof. Laurent Lessard </center>
#### <center> May 7<sup>th</sup>, 2018 </center>

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
  1. [Problem Description](#1.1.-Problem-Description)
  2. [Goals and Objectives](#1.2.-Goals-and-Objectives)
  3. [Report Outline](#1.3.-Report-Outline)
2. [Mathematical Model](#2.-Mathematical-Model)
  1. [Perfect Information](#2.1.-Perfect-Information)
    1. [Knowing Arrival Time & Duration](#2.1.1.-Knowing-Arrival-Time&-Duration)
    2. [Multicores](#2.1.2.-Multicores)
    3. [Process Dependency](#2.1.3.-Process-Dependency)
  2. [Robust Integer Programming](#2.2.-Robust-Programming)
    1. [Unknown Duration](#2.2.1.-Unknown-Duration)
    2. [Unknown Duration & Unknown Arrival Time](#2.2.2.-Unknown-Duration&-Unknown-Arrival-Time)
  3. [Stochastic](#2.3.-Stochastic)
    1. [Unknown Duration](#2.3.1.-Unknown-Duation)
    2. [Unknown Duration & Unknown Arrival Time](#2.3.2.-Unknown-Duration&-Unknown-Arrival-Time)
3. [Solution](#3.-Solution)
4. [Results and Discussion](#4.-Results-and-discussion)
5. [Conclusion](#5.-Conclusion)
6. [References](#6.-References)

*****

## 1. Introduction ##

### 1.1. Problem Description ###

Sitting in front of your personal computer, perhaps you are playing a video game, browsing a website, and having a chat with your friends via WeChat at the same time. How could it be possible to open so many applications on a computer and run them at the same time? Who is in charge here? The answer is the operating system.

<img src="operating_system.jpg" width="450" height="400" />
<center>Fig. 1: Operating system makes it possible to run multiple applications at the same time</center>

The **Operating System**(OS) is a really important piece of software present in virtually all computer systems, which manages computer hardware and softwares and provides common resources for computer programs. A computer program is a piece of code but it lives as a process when it is being executed. One of the duties of an operating system is to schedule processes to run on the computer. In 1950s, process scheduling was not an issue because there is not so much process so that the OS can serve all users/processes one by one. When it came to 1960s - early 1980s, multiprogramming and time-sharing evolves([history](https://en.wikipedia.org/wiki/Timeline_of_operating_systems)). Process scheduling should handle multiple processes, swap job to avoid idleness. There will be many processes that want to be executed. Time-sharing OS allows many users/processes to share the computing resources such as main memory, input/output(I/O) devices, files, etc.

The OS has many components which work together to help carry out resource management duties. For example there is a memory manager(manages the usage of the main memory), various device drivers(carry out low-level interactions with hardware devices), I/O manager(handles data transfer between memory and I/O devices), process manager(creates and destroys processes), file system manager(manages the file system), etc. Another important component is the process scheduler, also known as the **Central Processing Unit**(CPU), which is the focus of this project.

<!--![A shot of processes running on a multicore CPU and the resource usage](cpu_scheduling.png =100x20)-->
<img src="cpu_scheduling.png" width="540" height="270" />
<center>Fig. 2: A shot of processes running on a 4-core CPU and the resource usage</center>

The job of the scheduler is to assign tasks to CPUs, i.e. what is the next process to be run on which CPU. For a computer, there could be multiple CPUs and each CPU could have multiple cores. For a process, it could generate a number of threads to be executed at the same time. Since every core in a CPU could run its assigned task at the same time, we could seen a core as the most basic execution unit. In addition, since a thread is similar as a process(and in some OS they are treat nearly the same way) we could seen a thread as a process. Therefore, the number of processes that can be run at the same time($X$) can be expressed by the multiplication of the number of CPUs($C$) and the number of cores per CPU($c_{i}$):

$$
X = \sum_{i=1}^{C} c_i
$$

However, things are really complicated in reality. There can be mainly two kinds of processes, one is called CPU-bound process and the other is called I/O-bound process, which should be treated differently. There may also be dependencies or conflicts among different processes. A process cannot continue running without another process returning a result, or two processes are using the same resource so that they cannot be scheduled to run at the same time. In addition, scheduling among multicores brings more cost caused by **context switch**. When the ongoing process is being replaced by another process before it is done, its state information should be stored to somewhere else(main memory or hard disk) and the new process' state information(if exists) should be moved to CPU. This process is called context switch. Switching between different cores leads to more cost. Last but not least, a CPU cannot predict when a process will finish its execution and it is not possible for a CPU to know when a particular process will arrive.

Considering all the complexities, we will present a general model in this report to solve this problem and offer an efficient solution for scheduling processes in multicore CPU.

### 1.2. Goals and Objectives ###

How the CPU, the bottleneck device in the whole computer, is scheduled, has a huge impact on the performance of the computer system. To make better decisions, the following objectives have to be met:

- Minimize the total turnaround time: turnaround time is the period of time between the arrival of a process and the time it is finished, which should be as small as possible;
- Be fair to every process: there should not be any process being wait for a long time without being executed;
- Be as responsive as possible: a process should be treated soon after its arrival;

### 1.3. Report Outline ###

This report will divide the whole problem into multiple stages, progressiely solving more complex versions of this problem. Staring with something simple that we know the arrival time and the duraiton of each process(model 2.A.a.), then comes multicores(model 2.A.b.), then we deal with process dependencies(model 2.A.c.). After talking about models under perfect information, we will use two different models: robust programming(model 2.B.) and stochastic programming(model 2.C.) to deal with uncertainty in arrival time and duraiton of processes.

After presenting the models, we will give our solutions to each of the models and discuss about the result of the solutions. Different test cases and data can be come up with or drawn from the internet. Comparisons between our scheduling optimization and extant scheduling algorithms will also be made in the discussion section.

[shot]: https://github.com/tony-wj/Julia_hw/blob/master/cpu_scheduling.png

## 2. Mathematical model ##

A discussion of the modeling assumptions made in the problem (e.g. is it from physics? economics? something else?). Explain the decision variables, the constraints, and the objective function. Finally, show the optimization problem written in standard form. Discuss the model type (LP, QP, MIP, etc.). Equations should be formatted in $\LaTeX$ within the IJulia notebook. For this section you may **assume the reader is familiar with the material covered in class**.

Here is an example of an equation:

$$
\begin{bmatrix}
  1 & 2 \\
  3 & 4
\end{bmatrix}
\begin{bmatrix} x \\ y \end{bmatrix} =
\begin{bmatrix} 5 \\ 6 \end{bmatrix}
$$

And here is an example of an optimization problem in standard form:

$$
\begin{aligned}
\underset{x \in \mathbb{R^n}}{\text{maximize}}\qquad& f_0(x) \\
\text{subject to:}\qquad& f_i(x) \le 0 && i=1,\dots,m\\
& h_j(x) = 0 && j=1,\dots,r
\end{aligned}
$$

For some quick tips on using $\LaTeX$, see [this cheat sheet](http://users.dickinson.edu/~richesod/latex/latexcheatsheet.pdf).

## 3. Solution ##

Here, you should code up your model in Julia + JuMP and solve it. Your code should be clean, easy to read, well annotated and commented, and it should compile! You are not allowed to use other programming languages or DCP packages such as `convex.jl`. **I will be running your code**. I suggest having multiple code blocks separated by text blocks that explain the various parts of your solution. You may also solve several versions of your problem with different models/assumptions.

It's fine to call external packages such as `Gurobi`, but try to minimize the use of exotic libraries.

In [1]:
# this is a code block
using JuMP, Clp
m = Model(solver = ClpSolver())

things = [:horses, :donkeys, :goats]  # these are the things 
@variable(m, x[things] >= 0)          # the quantities of each of the things (can't be negative)
@constraint(m, sum(x) <= 10)          # we can't have any more than 10 things total
@objective(m, Max, x[:horses])        # we want to maximize the number of horses
solve(m)

for i in things
    println("The total number of ", i, " is: ", getvalue(x[i]))     # print result
end

The total number of horses is: 10.0
The total number of donkeys is: 0.0
The total number of goats is: 0.0


Remember to make sure your code compiles! I will be running your code!

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.

## 6. Reference ##

1. Operating Systems, Wikipedia, refer to https://en.wikipedia.org/wiki/Operating_system
2. Time-sharing, Wikipedia, refer to https://en.wikipedia.org/wiki/Time-sharing
3. History of CPU process scheduling, refer to http://dave-reed.com/csc539.S05/Lectures/CPUsched.ppt